<a href="https://colab.research.google.com/github/jang3463/machinelearning-deeplearning-lecture/blob/main/%ED%85%8D%EC%8A%A4%ED%8A%B8_%EB%B6%84%EC%84%9D%EC%9D%84_%EC%9C%84%ED%95%9C_%EC%9D%B8%EC%BD%94%EB%94%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 텍스트의 수치화

컴퓨터는 텍스트보다 숫자를 더 잘 처리한다.

- Integer Encoding (정수 인코딩)
  - 단어 토큰화 (Word Tokenization) 또는 형태소 분리 후 각 단어에 대한 **고유한 정수**를 부여
  - 중복이 허용되지 않는 모든 단어들의 집합을 만들어야 한다. **단어 집합(Vocabulary)**이라고 한다
- Padding
  - 모든 문장에 대해서 정수 인코딩을 수행했을 때 문장마다의 길이는 다를 수 있다.
  - 이 때 가상의 단어를 만들어 길이를 추가해 준다. (보통은 0으로 추가) 이를 padding 작업이라고 한다.
- Vectorization
  - One-Hot Encoding
    - 전체 단어 집합의 크기를 벡터의 차원으로 갖습니다.
    - 각 단어에 고유한 정수 인덱스를 부여하고, 해당 인덱스의 원소는 1, 나머지 원소는 0으로 가지는 벡터를 만든다.
  - Document Term Matrix (DTM)
    - 각 단어에 고유한 인덱스를 부여한 후에, 문서 마다 해당 단어가 등장한 횟수를 인덱스의 값으로 가진다.
  - TF-IDF (Term Frequency - Inverse Document Frequency)
    - 단어 빈도 - 역 문서 빈도
    - TF와 IDF라는 값을 곱한 값이다.
    - 문서의 유사도, 검색 시스템에서 검색 결과의 순위 등을 구하는 일에 사용된다.
    - 인공 신경망의 입력으로도 사용이 가능


임베딩에 대한 이해가 필수적이다

# Integer Encoding
문장을 단어 토큰화 또는 형태소 분리를 진행하면 각 단어 및 형태소들을 얻어 낼 수 있다. Integer Encoding은 아주 단순히 문장을 구성하는 단어들에 대해 **숫자를 부여** 했다고 생각하면 된다.

숫자를 부여하는 기준은 **ABC(가나다)**순, 또는 **빈도가 높은 순**으로 순서를 구성한다.

In [1]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
from nltk.tokenize import sent_tokenize
text = """Isn't she lovely.
Isn't she wonderful.
Isn't she precious.
Less than one minute old.
I never thought through love we'd be.
Making one as lovely as she.
But isn't she lovely made from love.
Isn't she pretty.
Truly the angel's best.
Boy, I'm so happy.
We have been heaven blessed.
I can't believe what God has done.
Through us he's given life to one.
But isn't she lovely made from love.
Isn't she lovely.
Life and love are the same.
Life is Aisha.
The meaning of her name.
Londie, it could have not been done.
Without you who conceived the one.
That's so very lovely made from love."""
text = sent_tokenize(text)
print(text)

["Isn't she lovely.", "Isn't she wonderful.", "Isn't she precious.", 'Less than one minute old.', "I never thought through love we'd be.", 'Making one as lovely as she.', "But isn't she lovely made from love.", "Isn't she pretty.", "Truly the angel's best.", "Boy, I'm so happy.", 'We have been heaven blessed.', "I can't believe what God has done.", "Through us he's given life to one.", "But isn't she lovely made from love.", "Isn't she lovely.", 'Life and love are the same.', 'Life is Aisha.', 'The meaning of her name.', 'Londie, it could have not been done.', 'Without you who conceived the one.', "That's so very lovely made from love."]


In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# 단어 토큰화된 전체 문장을 가지고 있을 배열
sentences = []
stop_words = set(stopwords.words('english')) # 불용어를 집합화

# 문장들을 하나씩 꺼내서 단어 토큰화
for i in text:
  sentence = word_tokenize(i) # 단어 토큰화
  result = [] # 정제 작업이 완료된 단어를 추가할 배열

  for word in sentence:
    # 소문자화
    word = word.lower()

    if word not in stop_words: # 불용어 처리
      # 길이가 2이하인 경우도 걸러준다.
      if len(word) > 2:
        result.append(word)

  sentences.append(result)

print(sentences)

[["n't", 'lovely'], ["n't", 'wonderful'], ["n't", 'precious'], ['less', 'one', 'minute', 'old'], ['never', 'thought', 'love'], ['making', 'one', 'lovely'], ["n't", 'lovely', 'made', 'love'], ["n't", 'pretty'], ['truly', 'angel', 'best'], ['boy', 'happy'], ['heaven', 'blessed'], ["n't", 'believe', 'god', 'done'], ['given', 'life', 'one'], ["n't", 'lovely', 'made', 'love'], ["n't", 'lovely'], ['life', 'love'], ['life', 'aisha'], ['meaning', 'name'], ['londie', 'could', 'done'], ['without', 'conceived', 'one'], ['lovely', 'made', 'love']]


## [English] 단어 집합 만들기 (python)

단어 집합이란? 중복을 제거한 단어들의 집합


In [5]:
# 각 배열에 등장한 횟수가 많을 수록 앞 번호에 위치
from collections import Counter
words = sum(sentences, []) # sentences에 들어있는 모든 배열을 비어있는 배열에 합치는 역할
print(words)

["n't", 'lovely', "n't", 'wonderful', "n't", 'precious', 'less', 'one', 'minute', 'old', 'never', 'thought', 'love', 'making', 'one', 'lovely', "n't", 'lovely', 'made', 'love', "n't", 'pretty', 'truly', 'angel', 'best', 'boy', 'happy', 'heaven', 'blessed', "n't", 'believe', 'god', 'done', 'given', 'life', 'one', "n't", 'lovely', 'made', 'love', "n't", 'lovely', 'life', 'love', 'life', 'aisha', 'meaning', 'name', 'londie', 'could', 'done', 'without', 'conceived', 'one', 'lovely', 'made', 'love']


In [8]:
# 단어 집합 만들기
vocab = Counter(words) # 파이썬의 Counter 모듈을 이용하면 단어의 모든 빈도를 쉽게 계산 가능
print(vocab)

Counter({"n't": 8, 'lovely': 6, 'love': 5, 'one': 4, 'made': 3, 'life': 3, 'done': 2, 'wonderful': 1, 'precious': 1, 'less': 1, 'minute': 1, 'old': 1, 'never': 1, 'thought': 1, 'making': 1, 'pretty': 1, 'truly': 1, 'angel': 1, 'best': 1, 'boy': 1, 'happy': 1, 'heaven': 1, 'blessed': 1, 'believe': 1, 'god': 1, 'given': 1, 'aisha': 1, 'meaning': 1, 'name': 1, 'londie': 1, 'could': 1, 'without': 1, 'conceived': 1})


In [9]:
# lovely가 몇 번 나왔는가?
vocab['lovely']

6

## [English] Integer Encoding (Python)

가나다 순 또는 빈도 내림차순으로 정렬하여 각 단어에 중복되지 않는 정수를 부여

In [11]:
# 빈도수가 높은 순서대로 정렬
vocab_sorted = sorted(vocab.items(), key = lambda x : x[1], reverse = True)
print(vocab_sorted)

[("n't", 8), ('lovely', 6), ('love', 5), ('one', 4), ('made', 3), ('life', 3), ('done', 2), ('wonderful', 1), ('precious', 1), ('less', 1), ('minute', 1), ('old', 1), ('never', 1), ('thought', 1), ('making', 1), ('pretty', 1), ('truly', 1), ('angel', 1), ('best', 1), ('boy', 1), ('happy', 1), ('heaven', 1), ('blessed', 1), ('believe', 1), ('god', 1), ('given', 1), ('aisha', 1), ('meaning', 1), ('name', 1), ('londie', 1), ('could', 1), ('without', 1), ('conceived', 1)]


In [12]:
# 높은 빈도수를 가진 단어일 수록 낮은 정수 인덱스를 부여 ( 앞쪽에 배치 )

word2idx = {}

i = 0

for (word,frequency) in vocab_sorted:
  if frequency > 1 : # 정제(Cleaning) 작업, 빈도수가 적은 단어는 제외
    i = i + 1
    word2idx[word] = i

print(word2idx)

{"n't": 1, 'lovely': 2, 'love': 3, 'one': 4, 'made': 5, 'life': 6, 'done': 7}


빈도수가 가장 높은 상위 n개만 선택해서 단어집합으로 사용하기

In [13]:
vocab_size = 5 # 단어 집합의 크기

# 단어 집합의 인덱스를 구해오기
words_frequency = [ w for w,c in word2idx.items() if c >= vocab_size+1 ]
print(words_frequency)

['life', 'done']


In [14]:
for w in words_frequency:
  del word2idx[w]

print(word2idx)

{"n't": 1, 'lovely': 2, 'love': 3, 'one': 4, 'made': 5}


### OOV (out of vocabulary) 처리

UNK (Unknown) 토큰을 추가할 것이다. UNK(OOV)란 단어집합에 없는 단어를 위한 인덱스 (신조어, 오타, 단어 집합에 포함되지 않는 단어를 위한 처리)

In [16]:
word2idx['UNK'] = 6
print(word2idx)

{"n't": 1, 'lovely': 2, 'love': 3, 'one': 4, 'made': 5, 'UNK': 6}


## [English] Encoding 해보기
- 단어를 숫자로 표현하는 것을 인코딩이라고 한다.

In [17]:
# 인코딩된 데이터를 저장하기 위한 배열
encoded = []

for s in sentences:
  # 임시로 인코딩된 내용을 저장할 배열
  temp = []

  # 단어 하나씩 꺼내기
  for w in s:
    # 있는 키라면
    try:
      # 단어 집합에서 꺼내서 임시 배열에 추가
      temp.append(word2idx[w])
    except KeyError:
      # 없는 키라면
      # 단어 집합에 없었던 단어이기 때문에 UNK 인코딩을 수행한다.
      temp.append(word2idx["UNK"])
  encoded.append(temp)

print("변환 전 : {}".format(sentences[:5]))
print("변환 후 : {}".format(encoded[:5]))

변환 전 : [["n't", 'lovely'], ["n't", 'wonderful'], ["n't", 'precious'], ['less', 'one', 'minute', 'old'], ['never', 'thought', 'love']]
변환 후 : [[1, 2], [1, 6], [1, 6], [6, 4, 6, 6], [6, 6, 3]]


## [English] Vocab & Integer Encoding (Tensorflow)


In [19]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()

# fit_on_texts() 함수에 코퍼스 (sentences)를 집어 넣으면 바로 빈도수를 기준으로 단어집합을 만들어 준다.
tokenizer.fit_on_texts(sentences)

print(tokenizer.word_index) # 생선된 단어집합 (vocab) 확인하기

{"n't": 1, 'lovely': 2, 'love': 3, 'one': 4, 'made': 5, 'life': 6, 'done': 7, 'wonderful': 8, 'precious': 9, 'less': 10, 'minute': 11, 'old': 12, 'never': 13, 'thought': 14, 'making': 15, 'pretty': 16, 'truly': 17, 'angel': 18, 'best': 19, 'boy': 20, 'happy': 21, 'heaven': 22, 'blessed': 23, 'believe': 24, 'god': 25, 'given': 26, 'aisha': 27, 'meaning': 28, 'name': 29, 'londie': 30, 'could': 31, 'without': 32, 'conceived': 33}


In [20]:
# 단어의 빈도수 확인하기
print(tokenizer.word_counts)

OrderedDict([("n't", 8), ('lovely', 6), ('wonderful', 1), ('precious', 1), ('less', 1), ('one', 4), ('minute', 1), ('old', 1), ('never', 1), ('thought', 1), ('love', 5), ('making', 1), ('made', 3), ('pretty', 1), ('truly', 1), ('angel', 1), ('best', 1), ('boy', 1), ('happy', 1), ('heaven', 1), ('blessed', 1), ('believe', 1), ('god', 1), ('done', 2), ('given', 1), ('life', 3), ('aisha', 1), ('meaning', 1), ('name', 1), ('londie', 1), ('could', 1), ('without', 1), ('conceived', 1)])


In [22]:
# 인코딩 수행하기
print(tokenizer.texts_to_sequences(sentences))

[[1, 2], [1, 8], [1, 9], [10, 4, 11, 12], [13, 14, 3], [15, 4, 2], [1, 2, 5, 3], [1, 16], [17, 18, 19], [20, 21], [22, 23], [1, 24, 25, 7], [26, 6, 4], [1, 2, 5, 3], [1, 2], [6, 3], [6, 27], [28, 29], [30, 31, 7], [32, 33, 4], [2, 5, 3]]


In [25]:
# 디코딩 수행하기
print(tokenizer.sequences_to_texts([[1,2],[1,9]]))

["n't lovely", "n't precious"]


## OOV 및 PADDING 설정
keras의 Tokenizer에 PADDING과 OOV를 사용하기로 했으면, PAD : 0 으로, OOV : 1 로 자동 설정된다.

In [27]:
# 상위 5개의 토큰만 사용하기
vocab_size = 5

tokenizer = Tokenizer(num_words=vocab_size+2, oov_token='OOV')
tokenizer.fit_on_texts(sentences)
print(tokenizer.texts_to_sequences(sentences))

[[2, 3], [2, 1], [2, 1], [1, 5, 1, 1], [1, 1, 4], [1, 5, 3], [2, 3, 6, 4], [2, 1], [1, 1, 1], [1, 1], [1, 1], [2, 1, 1, 1], [1, 1, 5], [2, 3, 6, 4], [2, 3], [1, 4], [1, 1], [1, 1], [1, 1, 1], [1, 1, 5], [3, 6, 4]]


In [28]:
print('oov 토큰의 인덱스 : {}'.format(tokenizer.word_index['OOV']))

oov 토큰의 인덱스 : 1


## [Korean] 토큰화 및 정수 인코딩을 Tensorflow로 구현

In [30]:
# !pip install konlpy

In [31]:
import pandas as pd
import numpy as np
import urllib.request
from tensorflow.keras.preprocessing.text import Tokenizer

In [44]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

('ratings_test.txt', <http.client.HTTPMessage at 0x7f4c013d65c0>)

In [45]:
train_data = pd.read_table('ratings_test.txt')
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        50000 non-null  int64 
 1   document  49997 non-null  object
 2   label     50000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.1+ MB


In [46]:
train_data.head()

id                                           document  label
0  6270596                                                굳 ㅋ      1
1  9274899                               GDNTOPCLASSINTHECLUB      0
2  8544678             뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아      0
3  6825595                   지루하지는 않은데 완전 막장임... 돈주고 보기에는....      0
4  6723715  3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??      0

1. null 값 제거
2. 중복제거
3. 특수문자 및 영어 제거

In [47]:
train_data['document'].nunique()

49157

In [48]:
# 중복 데이터 제거 및 null 값 제거
train_data.drop_duplicates(subset=['document'],inplace=True) # 중복 제거
train_data = train_data.dropna(how='any') # null값이 존재하는 행을 제거

In [49]:
# 정규식을 이용해서 한글만 추출
train_data['document'] = train_data['document'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', "" ) # 한글이 아닌 문자는 모두 " "
train_data.head()

id                                   document  label
0  6270596                                        굳 ㅋ      1
1  9274899                                                 0
2  8544678           뭐야 이 평점들은 나쁘진 않지만 점 짜리는 더더욱 아니잖아      0
3  6825595                  지루하지는 않은데 완전 막장임 돈주고 보기에는      0
4  6723715  만 아니었어도 별 다섯 개 줬을텐데 왜 로 나와서 제 심기를 불편하게 하죠      0

비어있는 문자열 정제

In [50]:
train_data['document'].replace('',np.nan,inplace=True)
train_data.isnull().sum()

id            0
document    162
label         0
dtype: int64

In [51]:
train_data = train_data.dropna(how='any')
print(len(train_data))

48995


In [54]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

보통 한국어에서의 불용어 처리는 stemming 또는 normalization 적용 후에 한다.

---



In [52]:
from konlpy.tag import Okt

okt = Okt()
okt.morphs('뭐야 이 평점들은 나쁘진 않지만 점 짜리는 더더욱 아니잖아',stem=True)

['뭐',
 '야',
 '이',
 '평점',
 '들',
 '은',
 '나쁘다',
 '않다',
 '점',
 '짜다',
 '리',
 '는',
 '더',
 '더욱',
 '아니다']

In [55]:
X_train = []

for sentence in train_data['document']:
  temp_X = []
  temp_X = okt.morphs(sentence, stem=True) # 토큰화
  temp_X = [w for w in temp_X if not w in stopwords] # 불용어 제거
  X_train.append(temp_X)

In [56]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [59]:
print(tokenizer.word_counts)

OrderedDict([('굳다', 137), ('ㅋ', 811), ('뭐', 1178), ('야', 508), ('평점', 2210), ('나쁘다', 218), ('않다', 2555), ('점', 2531), ('짜다', 318), ('리', 106), ('더', 1509), ('더욱', 94), ('아니다', 2667), ('지루하다', 1159), ('완전', 813), ('막장', 228), ('임', 702), ('돈', 696), ('주다', 1275), ('보기', 551), ('에는', 355), ('만', 2976), ('별', 634), ('다섯', 36), ('개', 698), ('왜', 1706), ('로', 2648), ('나오다', 2168), ('제', 491), ('심기', 1), ('불편하다', 103), ('음악', 390), ('주가', 16), ('되다', 3030), ('최고', 1950), ('영화', 17231), ('진정하다', 109), ('쓰레기', 1068), ('마치', 68), ('미국', 216), ('애니', 293), ('에서', 2330), ('튀어나오다', 19), ('창의력', 2), ('없다', 5308), ('로봇', 36), ('디자인', 19), ('부터가', 27), ('고개', 13), ('젖다', 21), ('갈수록', 189), ('개판', 59), ('중국영화', 28), ('유치하다', 386), ('내용', 1347), ('폼', 19), ('잡다', 119), ('끝나다', 477), ('말', 1445), ('안되다', 763), ('무기', 25), ('남무', 1), ('아', 1375), ('그리다', 217), ('동사서독', 2), ('같다', 2825), ('이건', 876), ('류', 209), ('류작', 24), ('이다', 4484), ('이별', 20), ('아픔', 40), ('뒤', 193), ('찾아오다', 17), ('새롭다', 147), ('인연

In [60]:
print(tokenizer.word_index)

{'영화': 1, '보다': 2, '을': 3, '없다': 4, '이다': 5, '있다': 6, '좋다': 7, '너무': 8, '다': 9, '정말': 10, '되다': 11, '재밌다': 12, '만': 13, '진짜': 14, '적': 15, '같다': 16, '아니다': 17, '로': 18, '않다': 19, '점': 20, '에서': 21, '만들다': 22, '평점': 23, '나오다': 24, '연기': 25, '것': 26, '내': 27, '최고': 28, '그': 29, '인': 30, '안': 31, '나': 32, '생각': 33, '게': 34, '못': 35, '왜': 36, '스토리': 37, '드라마': 38, '사람': 39, '감동': 40, '보고': 41, '때': 42, '이렇다': 43, '더': 44, '고': 45, '아깝다': 46, '말': 47, '감독': 48, '배우': 49, 'ㅋㅋ': 50, '아': 51, '그냥': 52, '내용': 53, '거': 54, '중': 55, '까지': 56, '재미있다': 57, '재미': 58, '시간': 59, '주다': 60, '요': 61, '자다': 62, '재미없다': 63, '뭐': 64, '하고': 65, '지루하다': 66, '가다': 67, '수': 68, '네': 69, '들다': 70, '쓰레기': 71, '사랑': 72, '그렇다': 73, '모르다': 74, '볼': 75, '싶다': 76, '작품': 77, '지': 78, '알다': 79, '다시': 80, '하나': 81, '마지막': 82, 'ㅠㅠ': 83, '이건': 84, '저': 85, '오다': 86, '정도': 87, '많다': 88, '완전': 89, '처음': 90, 'ㅋ': 91, '장면': 92, '주인공': 93, 'ㅋㅋㅋ': 94, '나다': 95, '이렇게': 96, '액션': 97, '안되다': 98, '차다': 99, '하': 100, '최악': 101, '걸': 